## Introduction:

- Any given coin has some probability, x, of landing heads up when spun on edge. 

- It seems reasonable to believe that the value of x depends on some physical characteristics of the coin, primarily the distribution of weight.

- If a coin is perfectly balanced, we expect x to be close to 50%, but for a lopsided coin, x might be substantially different. 

- We can use Bayes’s theorem and the observed data to estimate x.

The issue is that we wanna check if the coin is biased or not by checking weather it is fair or not .. 

### Steps:
1. Estimate the probability that the coin lands face up. 
2. Evaluate whether the data support the hypothesis that the coin is biased.

In [37]:
import euro
import thinkbayes
import matplotlib.pyplot as plt
% matplotlib inline 

In [2]:
euro.main()

Writing euro2.pdf
Writing euro2.eps
0.0209765261295
MLE 56
Mean 55.9523809524
Median 56
5th %ile 51
95th %ile 61
CI (51, 61)
0.0238475372147
MLE 56
Mean 55.7434994386
Median 56
5th %ile 51
95th %ile 61
CI (51, 61)
Writing euro1.pdf
Writing euro1.eps
Writing euro3.pdf
Writing euro3.eps


- Let's make the check ourselves step by step .. 

- Let’s define 101 hypotheses, where Hx is the hypothesis that the probability of heads is x%, for values from 0 to 100.
- Intially, the perior is uniform .. where all values have the same probability .. 
- The likelihood function is relatively easy ==> If Hx is true, the probability of heads is x/100 and the probability of tails is 1 − x/100.


In [3]:
dataset = 'H'*140 + 'T'*110

In [4]:
class Euro():
    suite = euro.UniformPrior()  # suite is 1001 elements 
    def Likelihood(self, data, hypo):
        x = hypo #x (heads) is out concern 
        if data == 'H':
            return x/100.0 #prob of heads
        else: 
            return 1 - (x/100.0)

In [5]:
Euro()

<__main__.Euro instance at 0x7f148d7f1f38>

In [6]:
euro.RunUpdate(Euro.suite)

- You can find the posterior distribution by finding the maximum likelihood function (i.e. the value with the highest probability) 

- Suite is the pmf .. hence, we can call Maximum likelihood function with it .. 

In [7]:
## it returns the value with the highest probability.

def MaximumLikelihood(pmf):
    prob, val = max((prob, val) for val, prob in pmf.Items())
    return val

In [8]:
MaximumLikelihood(Euro.suite)

56

Make sense ! :D 
    
the result is 56, which is also the observed percentage of heads, 140/250 = 56%. 

In [9]:
## mean, median and CI ..

Euro.suite.Mean(), thinkbayes.Percentile(Euro.suite, 50), thinkbayes.CredibleInterval(Euro.suite, 90)

(55.952380952380956, 56, (51, 61))

the posterior credible interval does not include 50%, which suggests that the coin is not fair !!

### wait a minute .. 

we are not interested in checking whether the coin is fair or not .. but biased or not .. 
hence, it might be tempting to ask for the probability that x is 50%

In [10]:
Euro.suite.Prob(50)

0.02097652612954468

 almost meaningless value !
 
 Thus, we could have divided the range into more or fewer pieces, and if we had, the probability for any given hypothesis would be greater or less.

## Swapping the perior:

Let's try other perior probabilities instead of uniform .. cause it doesn't make sense to use uniform parior if the coin is lopsided (prob. of each side is not 50% but 10% or 90%) .. 

Hence, we gonna choose a prior that gives higher prob. near 50% and lower at the extremes .. 

In [33]:
def Traingle_perior():
    suite = euro.Euro()
    
    for i in range(0, 51):
        suite.Set(i, i)
    for i in range(52, 101):    
        suite.Set(i, 100 - i)
    suite.Normalize()
    
    return suite

In [34]:
s = Traingle_perior()
s

In [46]:
euro.RunUpdate(s)
euro.Summarize(s)

6.66022198907e-09
MLE 56
Mean 55.9728405408
Median 56
5th %ile 54
95th %ile 58
CI (54, 58)


Swamping the priors means -with enough data- people who start with different priors will tend to converge on the same posterior.